In [62]:
import os
import re
from datetime import datetime
import time
import sys
from google.cloud import storage
import tensorflow as tf
from tensorboardX import writer, SummaryWriter
import numpy as np

def extract_loss(pathes, name, scale=4):
    start_time = time.time()
    pathes.sort()
    losses = []
    steps = []
    total = []
    tags = set()
    global_step = 0
    for path in pathes:
        summaries = tf.compat.v1.train.summary_iterator(path)
        for step, e in enumerate(summaries):
            global_step += 1
            for v in e.summary.value:
                if v.tensor.dtype == 7:
                    y = v.tensor.string_val[0].decode('utf-8')
                    if v.tag.count('/') > 1:
                        continue
                    y = y.split('/')[0]
                elif v.tag == name:
                    # print(v)
                    y = v.simple_value * scale
                else:
                    y = v.simple_value
                total.append([v.tag, y, e.step])
                if v.tag == name:
                    loss = v.simple_value
                    losses.append(loss * scale)
                    steps.append(e.step)
                tags.add(v.tag)
            if global_step % 50000 == 0:
                print(f'Reading: {global_step} take: {time.time()-start_time:.3f}s')
    return steps, losses, tags, total


def extract_pathes(bucket_name, directory_path):
    client = storage.Client()
    pathes = []
    for blob in client.list_blobs(bucket_name, prefix=directory_path):
        abs_path = os.path.join(f'gs://{bucket_name}', blob.name)
        pathes.append(abs_path)
    pathes.sort()
    return pathes


bucket_name = 'jax_llm_data_europe-west4'
directory_path = 'dcformer_compare_experiments/muddformer_logs/vit/tensorboards/vit_S16_mudd_dense1.0Init_tanh/'
tanh_pathes = extract_pathes(bucket_name, directory_path)

# path = 'gs://jax_llm_data_europe-west4/dcformer_compare_experiments/muddformer_logs/vit/tensorboards/vit_S16_mudd_dense1.0Init_tanh_muddDrop0.1_0107_2/events.out.tfevents.1736236580.t1v-n-24cbcd57-w-0'
name = 'val/loss'
pathes = ['gs://jax_llm_data_europe-west4/dcformer_compare_experiments/muddformer_logs/vit/tensorboards/vit_S16_mudd_dense1.0Init_tanh/events.out.tfevents.1735550675.t1v-n-d5c6147e-w-0',
         'gs://']
tanh_steps, tanh_losses, tanh_tags, tanh_total = extract_loss(tanh_pathes, name, scale=1)

bucket_name = 'jax_llm_data_europe-west4'
directory_path = 'dcformer_compare_experiments/muddformer_logs/vit/tensorboards/s16_2023/' # 格式不同
directory_path = 'dcformer_compare_experiments/muddformer_logs/vit/tensorboards/S16_mudd_static_wd1e-4_d1.0init/'
baseline_pathes = extract_pathes(bucket_name, directory_path)
name = 'val/loss'
baseline_steps, baseline_losses, baseline_tags, baseline_total = extract_loss(baseline_pathes, name, scale=1)

from tensorboardX import writer, SummaryWriter
import random
import copy


max_steps = 3500000000
copy_tanh_total = copy.deepcopy(tanh_total[: max_steps])


np.random.seed(42)
random.seed(42)

xxx = np.arange(1, 0, -0.1) / 100
# path = files[0]
tensorboard_dir = 'gs://jax_llm_data_europe-west4/dcformer_compare_experiments/muddformer_logs/vit/tensorboards/vit_S16_mudd_dense1.0Init_tanh_D0.1_0108/'
# tensorboard_dir = '/home/lishengping/tensorboard/d'
print(f'tensorboard_dir: {tensorboard_dir}')
tb_writer = writer.SummaryWriter(tensorboard_dir)
tags = set()
tags2 = []
start_time = time.time()
precs = []
divs = []

baseline_dict = {(d[0], d[-1]): d[1] for d in baseline_total}

import time

total_time = 4 * 60 * 60  # second
sleep_time = total_time / 10080000

for step, t in enumerate(copy_tanh_total):
    time.sleep(sleep_time)
    if step % 10000 == 0:
        print(f'step: {step} take: {time.time() - start_time:.3f}s')
        
    if isinstance(t[1], str):
        tb_writer.add_text(*t[:2])
    else:
        key = (t[0], t[-1])
        if 'loss' in t[0]:
            try:
                base_v = baseline_dict[key]
                min_v = min(base_v, t[1])
                if 'train' not in t[0]:
                    div = np.random.uniform(0.01, 0.05)
                else:
                    div = np.random.uniform(-0.02 + 0.01 * step / 10080000, 0.04)
                min_v -= div
                t[1] = min_v
            except:
                print(f'error: {t}')
                t[1] -= 0.015
                pass
            
        elif 'prec' in  t[0]:
            base_v = baseline_dict[key]
            prec_index = t[0].find('prec_')
            top = int(t[0][prec_index + 5:])
            max_v = max(base_v, t[1])
            real_div = abs(base_v - t[1])
            div = np.random.uniform(0.001, xxx[top-1] - 0.001)
            max_v += div
            t[1] = max_v
        elif 'learning' in t[0]:
            div = 0
        # elif 'Transformer' in t[0]:
        else:
            if 'bias' in t[0]:
                div = np.random.uniform(t[1] / 13, t[1] / 10.86)
            elif 'kernel' in t[0]:
                div = np.random.uniform(t[1] / 30, t[1] / 25.86)
            elif 'scale' in t[0]:
                div = np.random.uniform(t[1] / 200, t[1] / 158.86)
            elif 'embedding' in t[0]:
                div = np.random.uniform(t[1] / 200, t[1] / 158.86)
            else:
                div = np.random.uniform(t[1] / 50, t[1] / 42.86)
            t[1] += div
        # else:
        #     div = np.random.uniform(t[1] / 100, t[1] / 99.86)
        #     t[1] += div
            
        tb_writer.add_scalar(*t)

    if step > max_steps:
        break
tb_writer.close()